In [9]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from src.data.data_fetcher import get_raw_data
from src.features.feature_engineering import create_time_features_from_date
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

from src.data.data_fetcher import get_all_features, get_raw_data
from src.features.feature_engineering import prepare_data
from src.features.preprocess_data import get_preprocessed_test_data, fetch_preprocessed_data
pd.set_option('display.max_columns', 200)


from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')


%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\gunna\Documents\Maskinlæring\Prosjekt\power-predictor\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [10]:
targets_A, targets_B, targets_C, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()
all_features = get_raw_data()


train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
x_test_whole = get_preprocessed_test_data()

x_whole = pd.concat([X_train_obs_combined, X_val_obs_combined])
y_whole = pd.concat([y_train_obs_combined, y_val_obs_combined])
x_whole.reset_index(drop=True, inplace=True)
y_whole.reset_index(drop=True, inplace=True)

x_whole.head()

X_test_estimated_a.shape = (2880, 47), X_test_estimated_b.shape = (2880, 47), X_test_estimated_c.shape = (2880, 47)
After temporal alignment
X_test_estimated_a.shape = (720, 47), X_test_estimated_b.shape = (720, 47), X_test_estimated_c.shape = (720, 47)
X_test_estimated_a_processed.shape = (720, 46), X_test_estimated_b_processed.shape = (720, 46), X_test_estimated_c_processed.shape = (720, 46)


,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,is_day:idx,is_in_shadow:idx,precip_5min:mm,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sun_azimuth:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location_a,location_b,location_c,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,sun_product,modified_solar_elevation,effective_radiation,time_since_prediction,cloud_ratio,cloud_cover_over_30%,sun_addition,is_freezing,is_snow,is_rain
0,2020-11-21 16:00:00,4.675,1.23925,0.000000e+00,0.000000,2067.925049,0.0,272.924988,0.000000,0.000000,0.000000,0.000000,91.949997,0.0,1.0,0.0,0.0,981.574951,0.0,0.0,65.949997,252.160492,0.0,279.774994,98.974998,45661.750000,3.800,1.100,3.625,0.0,1,0,0,-0.638384,0.769718,-0.866025,-0.500000,0.000000,0.000000,0.000000,0,0.929022,1,0.000000,0,0,0
1,2019-12-14 10:00:00,4.400,1.23625,4.938225e+04,20.875000,1698.875000,0.0,271.975006,19.175001,43152.023438,2.250000,4051.275146,73.699997,1.0,0.0,0.0,0.0,976.950012,0.0,0.0,70.324997,168.481506,0.0,278.000000,93.649994,54169.699219,3.850,-3.775,0.600,0.0,0,1,0,-0.308812,0.951123,0.500000,-0.866025,43.143753,0.048057,0.082039,0,0.786973,1,21.425001,0,0,0
2,2021-08-13 04:00:00,11.350,1.21775,9.830708e+04,58.775002,124.950005,0.0,286.299988,32.450001,65975.726562,1.000000,1871.175049,86.474998,1.0,0.0,0.0,0.0,1005.925049,0.0,0.0,90.275002,69.381248,0.0,287.549988,99.900002,33840.726562,1.825,1.600,0.825,0.0,0,1,0,-0.653160,-0.757220,0.866025,0.500000,32.450001,0.108581,0.019034,0,0.865616,1,33.450001,0,0,0
3,2022-05-04 00:00:00,4.150,1.30050,0.000000e+00,0.000000,5375.950195,0.0,271.125000,0.000000,0.000000,0.000000,0.000000,64.849998,0.0,1.0,0.0,0.0,1012.450012,0.0,0.0,78.275002,15.757500,0.0,273.549988,95.625000,28736.724609,1.950,-0.550,1.850,0.0,1,0,0,0.855075,-0.518505,0.000000,1.000000,0.000000,0.000000,0.000000,0,0.678170,1,0.000000,0,0,0
4,2019-04-03 08:00:00,4.300,1.25400,1.162696e+06,378.500000,1466.699951,0.0,271.675018,87.324997,286677.937500,278.799988,848206.062500,24.150000,1.0,0.0,0.0,0.0,1003.000000,0.0,0.0,52.099998,130.110260,0.0,278.500000,48.250000,42909.300781,1.000,-0.775,0.675,0.0,0,1,0,0.999930,-0.011826,0.866025,-0.500000,24346.208984,0.394620,0.729517,0,0.500518,0,366.125000,0,0,0


In [11]:
x_test_whole.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,is_day:idx,is_in_shadow:idx,precip_5min:mm,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sun_azimuth:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location_a,location_b,location_c,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,sun_product,modified_solar_elevation,effective_radiation,time_since_prediction,cloud_ratio,cloud_cover_over_30%,sun_addition,is_freezing,is_snow,is_rain
0,4.325,1.28675,0.000000,0.000,1061.550049,0.0,271.650024,0.000000,0.000000,0.000000,0.000000,74.949997,0.00,1.0,0.0,0.0,1006.799988,0.0,0.0,80.275002,16.026501,0.0,273.799988,74.949997,29907.500000,3.950,2.100,3.35,0.0,1,0,0,0.880683,-0.473706,0.000000,1.000000,0.000000,0.000000,0.000000,16.998889,1.0,1,0.000000,0,0,0
1,4.275,1.28600,0.000000,0.000,1075.100098,0.0,271.450012,0.000000,0.000000,0.000000,0.000000,77.474998,0.00,1.0,0.0,0.0,1006.299988,0.0,0.0,79.824997,30.497250,0.0,273.799988,77.474998,29519.074219,3.825,1.925,3.30,0.0,1,0,0,0.880683,-0.473706,0.258819,0.965926,0.000000,0.000000,0.000000,17.998889,1.0,1,0.000000,0,0,0
2,4.150,1.28375,0.000000,0.000,1200.400024,0.0,271.049988,0.000000,0.000000,0.000000,0.000000,88.099998,0.00,1.0,0.0,0.0,1005.799988,0.0,0.0,78.000000,44.517250,0.0,273.849976,88.099998,31009.125000,3.650,1.750,3.20,0.0,1,0,0,0.880683,-0.473706,0.500000,0.866025,0.000000,0.000000,0.000000,18.998889,1.0,1,0.000000,0,0,0
3,4.025,1.28200,10124.424805,11.675,1179.849976,0.0,270.649994,9.375000,16845.226562,2.100000,3765.350098,68.599998,0.75,0.5,0.0,0.0,1005.224976,0.0,0.0,75.625000,58.083000,0.0,273.899994,68.599998,34552.500000,3.500,1.450,3.15,0.0,1,0,0,0.880683,-0.473706,0.707107,0.707107,19.687500,0.024650,0.371908,19.998889,1.0,1,11.475000,0,0,0
4,3.900,1.28100,141748.593750,76.875,920.049988,0.0,270.375000,47.400002,102209.703125,25.450001,49571.199219,66.300003,1.00,0.0,0.0,0.0,1004.750000,0.0,0.0,74.224998,71.341003,0.0,273.924988,66.300003,35483.875000,3.325,1.300,3.05,0.0,1,0,0,0.880683,-0.473706,0.866025,0.500000,1206.330078,0.129981,0.349712,20.998889,1.0,1,72.850006,0,0,0


In [12]:
X_train_observed_a['date_forecast'] = pd.to_datetime(X_train_observed_a['date_forecast'])

In [13]:
# Check the range of indices in both dataframes
range_targets_A = (targets_A.index.min(), targets_A.index.max())
range_X_train_observed_a = (X_train_observed_a.index.min(), X_train_observed_a.index.max())

print("Range of indices in targets_A:", range_targets_A)
print("Range of indices in X_train_observed_a:", range_X_train_observed_a)


Range of indices in targets_A: (0, 34084)
Range of indices in X_train_observed_a: (0, 118668)


In [14]:
# Load the dataset for Location A
targets_A['time'] = pd.to_datetime(targets_A['time'])
targets_A.set_index('time', inplace=True)

targets_B['time'] = pd.to_datetime(targets_B['time'])
targets_B.set_index('time', inplace=True)

targets_C['time'] = pd.to_datetime(targets_C['time'])
targets_C.set_index('time', inplace=True)


In [15]:
targets_A.columns

Index(['pv_measurement'], dtype='object')

In [20]:
import matplotlib.pyplot as plt

def plot_average_pv_using_date_forecast(targets_df, x_train_observed_df, location_name):
    # Ensure 'date_forecast' is in datetime format in both DataFrames

    # Now proceed with the merge
    combined_df = pd.merge(targets_df, x_train_observed_df, left_index=True, right_on='date_forecast')
    
    # Extracting Hour Information from the 'date_forecast' column
    combined_df["hour"] = combined_df['date_forecast'].dt.hour

    # Grouping and Averaging
    average_pv_by_hour = combined_df.groupby("hour")["pv_measurement"].mean()

    print(average_pv_by_hour)

    # Plotting
    # average_pv_by_hour.plot(kind="bar", figsize=(10, 6), title=f"Average PV Measurement for Each Hour of the Day - {location_name}")
    # plt.xlabel("Hour of the Day")
    # plt.ylabel("Average PV Measurement")
    # plt.grid(axis="y")
    # plt.tight_layout()
    # plt.show()

# For representation purposes, the function's return value
"Function defined. You can now run this function in your environment for the desired results."


# Extract months from the datetime index
months = targets_A.index.month

# Filter rows where the month is between April (4) and July (7)
filtered_df = targets_A[(months >= 4) & (months <= 7)]
# Call the function for each location's dataframes
plot_average_pv_using_date_forecast(filtered_df, X_train_observed_a, "Location A")
# plot_average_pv_using_date_forecast(filtered_df, X_train_observed_b, "Location B")
# plot_average_pv_using_date_forecast(y_whole, x_whole, "Location C")

hour
0        0.009318
1        3.275153
2       37.997365
3      137.552800
4      370.981082
5      813.822447
6     1414.227812
7     2017.630353
8     2553.940141
9     2892.494259
10    3041.032565
11    3073.337365
12    2888.446776
13    2624.679718
14    2215.669176
15    1766.465412
16    1172.741318
17     630.691859
18     297.967482
19     132.128376
20      32.066682
21       1.135718
22       0.001549
23       0.000000
Name: pv_measurement, dtype: float64


In [24]:
print(x_test_whole.isna().sum())


absolute_humidity_2m:gm3          0
air_density_2m:kgm3               0
clear_sky_energy_1h:J             0
clear_sky_rad:W                   0
cloud_base_agl:m                  0
dew_or_rime:idx                   0
dew_point_2m:K                    0
diffuse_rad:W                     0
diffuse_rad_1h:J                  0
direct_rad:W                      0
direct_rad_1h:J                   0
effective_cloud_cover:p           0
is_day:idx                        0
is_in_shadow:idx                  0
precip_5min:mm                    0
precip_type_5min:idx              0
pressure_50m:hPa                  0
prob_rime:p                       0
rain_water:kgm2                   0
relative_humidity_1000hPa:p       0
sun_azimuth:d                     0
super_cooled_liquid_water:kgm2    0
t_1000hPa:K                       0
total_cloud_cover:p               0
visibility:m                      0
wind_speed_10m:ms                 0
wind_speed_u_10m:ms               0
wind_speed_v_10m:ms         

In [ ]:
def calculate_pv_percentage_by_hour(targets_df, x_train_observed_df):
    # Ensure the 'date_forecast' column is a Datetime column
    x_train_observed_df['date_forecast'] = pd.to_datetime(x_train_observed_df['date_forecast'])

    # Merging the DataFrames using 'date_forecast' for x_train_observed_df and index for targets_df
    combined_df = pd.merge(targets_df, x_train_observed_df, left_index=True, right_on='date_forecast')
    
    # Extracting Hour Information from the 'date_forecast' column
    combined_df["hour"] = combined_df['date_forecast'].dt.hour

    # Grouping and Averaging
    average_pv_by_hour = combined_df.groupby("hour")["pv_measurement"].mean()

    # Calculate the percentage of the max hour
    max_hour_value = average_pv_by_hour.max()
    percentage_of_max_hour = (average_pv_by_hour / max_hour_value) * 100

    return percentage_of_max_hour

# For representation purposes, the function's return value
"Function defined. You can now run this function in your environment for the desired results."



'Function defined. You can now run this function in your environment for the desired results.'

In [ ]:
percentage_A = calculate_pv_percentage_by_hour(targets_A, X_train_observed_a)
percentage_B = calculate_pv_percentage_by_hour(targets_B, X_train_observed_b)
percentage_C = calculate_pv_percentage_by_hour(targets_C, X_train_observed_c)

print("Location A - Percentage of Max Hour PV:")
print(percentage_A)
print("\nLocation B - Percentage of Max Hour PV:")
print(percentage_B)
print("\nLocation C - Percentage of Max Hour PV:")
print(percentage_C)


Location A - Percentage of Max Hour PV:
hour
0       0.000159
1       0.056063
2       0.652588
3       2.493848
4       7.398189
5      17.937975
6      34.967078
7      54.989824
8      74.842756
9      89.825527
10     98.383528
11    100.000000
12     91.833482
13     77.776764
14     60.984522
15     44.170774
16     27.015991
17     13.350792
18      5.895078
19      2.387916
20      0.549216
21      0.019441
22      0.000027
23      0.000000
Name: pv_measurement, dtype: float64

Location B - Percentage of Max Hour PV:
hour
0       4.961233
1       5.281501
2       6.013889
3       8.548226
4      14.711482
5      24.720872
6      39.861623
7      57.147685
8      75.389541
9      88.561434
10     98.307949
11    100.000000
12     93.088619
13     79.528733
14     62.269500
15     46.851170
16     31.756596
17     19.487354
18     12.163215
19      7.510654
20      5.419775
21      5.068415
22      4.931610
23      4.979176
Name: pv_measurement, dtype: float64

Location C - Perce

In [ ]:
X_train_observed_a.columns

Index(['date_forecast', 'absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'dew_or_rime:idx', 'dew_point_2m:K',
       'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx',
       'snow_melt_10min:mm', 'snow_water:kgm2', 'sun_azimuth:d',
       'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'wind_speed_u_10m:ms',

train_a.isna()

In [ ]:
X_train_observed_a.head()

value_counts = X_train_observed_a['Rain'].value_counts().sort_index()
value_counts

KeyError: 'Rain'

In [ ]:
# Combine all the datasets except for train_a, train_b, and train_c
combined_df = pd.concat([
    X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, 
    X_train_observed_a, X_train_observed_b, X_train_observed_c, 
    X_test_estimated_a, X_test_estimated_b, X_test_estimated_c
])

# Calculate the number of NaN values for each column
na_count_per_column = combined_df.isna().sum()

print(na_count_per_column)

combined_df.shape

date_calc                         352423
date_forecast                          0
absolute_humidity_2m:gm3               0
air_density_2m:kgm3                    0
ceiling_height_agl:m               75543
clear_sky_energy_1h:J                  0
clear_sky_rad:W                        0
cloud_base_agl:m                   31862
dew_or_rime:idx                        0
dew_point_2m:K                         0
diffuse_rad:W                          0
diffuse_rad_1h:J                       0
direct_rad:W                           0
direct_rad_1h:J                        0
effective_cloud_cover:p                0
elevation:m                            0
fresh_snow_12h:cm                      0
fresh_snow_1h:cm                       0
fresh_snow_24h:cm                      0
fresh_snow_3h:cm                       0
fresh_snow_6h:cm                       0
is_day:idx                             0
is_in_shadow:idx                       0
msl_pressure:hPa                       0
precip_5min:mm  

(413791, 47)

In [ ]:
X_train_observed_a.shape

(118669, 46)

In [ ]:
X_train_observed_c.isna().sum()

date_forecast                          0
absolute_humidity_2m:gm3               0
air_density_2m:kgm3                    0
ceiling_height_agl:m               19923
clear_sky_energy_1h:J                  0
clear_sky_rad:W                        0
cloud_base_agl:m                    8512
dew_or_rime:idx                        0
dew_point_2m:K                         0
diffuse_rad:W                          0
diffuse_rad_1h:J                       0
direct_rad:W                           0
direct_rad_1h:J                        0
effective_cloud_cover:p                0
elevation:m                            0
fresh_snow_12h:cm                      0
fresh_snow_1h:cm                       0
fresh_snow_24h:cm                      0
fresh_snow_3h:cm                       0
fresh_snow_6h:cm                       0
is_day:idx                             0
is_in_shadow:idx                       0
msl_pressure:hPa                       0
precip_5min:mm                         0
precip_type_5min

In [ ]:
df.isna().sum()


NameError: name 'df' is not defined

In [ ]:
print(f"train_a: {train_a.isna().sum()}")
print(f"train_b: {train_b.isna().sum()}")
print(f"train_c: {train_c.isna().sum()}")
print(f"X_train_estimated_a: {X_train_estimated_a.isna().sum()}")
print(f"X_train_estimated_b: {X_train_estimated_b.isna().sum()}")
print(f"X_train_estimated_c: {X_train_estimated_c.isna().sum()}")
print(f"X_train_observed_a: {X_train_observed_a.isna().sum()}")
print(f"X_train_observed_b: {X_train_observed_b.isna().sum()}")
print(f"X_train_observed_c: {X_train_observed_c.isna().sum()}")
print(f"X_test_estimated_a: {X_test_estimated_a.isna().sum()}")
print(f"X_test_estimated_b: {X_test_estimated_b.isna().sum()}")
print(f"X_test_estimated_c: {X_test_estimated_c.isna().sum()}")

train_a: time              0
pv_measurement    0
dtype: int64
train_b: time              0
pv_measurement    4
dtype: int64
train_c: time                 0
pv_measurement    6060
dtype: int64
X_train_estimated_a: date_calc                             0
date_forecast                         0
absolute_humidity_2m:gm3              0
air_density_2m:kgm3                   0
ceiling_height_agl:m               3919
clear_sky_energy_1h:J                 0
clear_sky_rad:W                       0
cloud_base_agl:m                   2094
dew_or_rime:idx                       0
dew_point_2m:K                        0
diffuse_rad:W                         0
diffuse_rad_1h:J                      0
direct_rad:W                          0
direct_rad_1h:J                       0
effective_cloud_cover:p               0
elevation:m                           0
fresh_snow_12h:cm                     0
fresh_snow_1h:cm                      0
fresh_snow_24h:cm                     0
fresh_snow_3h:cm           